In [3]:
from __future__ import print_function
from awips.dataaccess import DataAccessLayer
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
from shapely.ops import cascaded_union


In [4]:
# Server, Data Request Type, and Database Table
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest('maps')
request.addIdentifier('table', 'mapdata.county')

In [6]:
# Define a WFO ID for location
# tie this ID to the mapdata.county column "cwa" for filtering
request.setLocationNames('BOU')
request.addIdentifier('cwa', 'BOU')

# enable location filtering (inLocation)
# locationField is tied to the above cwa definition (BOU)
request.addIdentifier('geomField', 'the_geom')
request.addIdentifier('inLocation', 'true')
request.addIdentifier('locationField', 'cwa')

# This is essentially the same as "'"select count(*) from mapdata.cwa where cwa='BOU';" (=1)

# Get response and create dict of county geometries
response = DataAccessLayer.getGeometryData(request, [])
counties = np.array([])
for ob in response:
    counties = np.append(counties,ob.getGeometry())
print("Using " + str(len(counties)) + " county MultiPolygons")


%matplotlib inline
# All WFO counties merged to a single Polygon
merged_counties = cascaded_union(counties)
envelope = merged_counties.buffer(2)
boundaries=[merged_counties]

ThriftRequestException: b'com.raytheon.uf.common.dataaccess.exception.UnsupportedOutputTypeException' exception thrown: b'topo does not support geometry data'
	at com.raytheon.uf.common.dataaccess.impl.AbstractDataFactory.getGeometryData(AbstractDataFactory.java:250)
	at com.raytheon.uf.common.dataaccess.DataAccessLayer.getGeometryData(DataAccessLayer.java:160)
	at com.raytheon.uf.edex.dataaccess.handler.GetGeometryDataHandler.handleRequest(GetGeometryDataHandler.java:70)
	at com.raytheon.uf.edex.dataaccess.handler.GetGeometryDataHandler.handleRequest(GetGeometryDataHandler.java:1)
	at com.raytheon.uf.edex.requestsrv.RequestServiceExecutor.execute(RequestServiceExecutor.java:155)
	at com.raytheon.uf.edex.requestsrv.serialization.SerializingStreamExecutor.execute(SerializingStreamExecutor.java:107)
	at com.raytheon.uf.edex.requestsrv.serialization.ByteLimitingStreamExecutor.execute(ByteLimitingStreamExecutor.java:76)
	at com.raytheon.uf.edex.requestsrv.http.HttpRequestServiceExecutor.ex

In [5]:
request = DataAccessLayer.newDataRequest("topo")
request.addIdentifier("group", "/")
request.addIdentifier("dataset", "full")
request.setEnvelope(envelope)
gridData = DataAccessLayer.getGridData(request)
print(gridData)
print("Number of grid records: " + str(len(gridData)))
print("Sample grid data shape:\n" + str(gridData[0].getRawData().shape) + "\n")
print("Sample grid data:\n" + str(gridData[0].getRawData()) + "\n")

NameError: name 'envelope' is not defined

In [ ]:
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature,NaturalEarthFeature
import cartopy.crs as ccrs